### Import libraries

In [2]:
import pandas as pd
import numpy as np
#!pip install beautifulsoup4
from bs4 import BeautifulSoup
import requests
#!pip install lxml
import lxml

     |████████████████████████████████| 112kB 22.9MB/s eta 0:00:01
     |████████████████████████████████| 5.8MB 22.6MB/s eta 0:00:01


### Getting the data from wikipedia

In [3]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = df[0]
df.head(20)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Etobicoke,Islington Avenue


**Ignore cells with a borough that is Not assigned.**

In [4]:
df = df.drop(df[(df.Borough == "Not assigned")].index)
# give "Not assigned" Neighborhoods same name as Borough:
df.Neighbourhood.replace("Not assigned", df.Borough, inplace=True)
# copy Borough value to Neighborhood if NaN:
df.Neighbourhood.fillna(df.Borough, inplace=True)
# drop duplicate rows:
df=df.drop_duplicates()
df.head(20)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Etobicoke,Islington Avenue
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [5]:
#  combine multiple neighborhoods with the same post code
df_cleaned = df.groupby("Postcode").agg({'Borough': 'first',
                                             'Neighbourhood': ', '.join}).reset_index()
df_cleaned.head(20), df_cleaned.shape

(   Postcode      Borough                                      Neighbourhood
 0       M1B  Scarborough                                     Rouge, Malvern
 1       M1C  Scarborough             Highland Creek, Rouge Hill, Port Union
 2       M1E  Scarborough                  Guildwood, Morningside, West Hill
 3       M1G  Scarborough                                             Woburn
 4       M1H  Scarborough                                          Cedarbrae
 5       M1J  Scarborough                                Scarborough Village
 6       M1K  Scarborough        East Birchmount Park, Ionview, Kennedy Park
 7       M1L  Scarborough                    Clairlea, Golden Mile, Oakridge
 8       M1M  Scarborough    Cliffcrest, Cliffside, Scarborough Village West
 9       M1N  Scarborough                        Birch Cliff, Cliffside West
 10      M1P  Scarborough  Dorset Park, Scarborough Town Centre, Wexford ...
 11      M1R  Scarborough                                  Maryvale, Wexford

Question 2

In [6]:

df_cleaned.rename(columns={'Postcode':'Postal Code'},inplace=True)
df_cleaned.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
#Opening the provided CSV file into a dataframe
df_add = pd.read_csv("http://cocl.us/Geospatial_data")
df_add.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
#Combind the dataframes so that the matching postcode have the corresponding coordinates
df=pd.merge(df_cleaned, df_add, on='Postal Code')
df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [9]:
df.shape

(103, 5)

### Question 3

**Filtering for Boroughs with the word Toronto**

In [11]:
df_toronto = df[df['Borough'].str.contains('Toronto')]
df_toronto.reset_index(drop=True, inplace=True)
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
